### Emulate USB serial port on terminal
* terminal 1
```bash
sudo socat PTY,link=/dev/ttyS10 PTY,link=/dev/ttyS11
```
* terminal 2
```bash
sudo chown $USER /dev/ttyS10 \
&& sudo chown $USER /dev/ttyS11
```

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
from pkg.utils.utils import *

print_com_ports()

Connected COM ports: ['/dev/ttyS11', '/dev/ttyS10', '/dev/ttyUSB0']


In [2]:
import serial
import serial.tools.list_ports as sp
import numpy as np
import time
from enum import Enum
from ctypes import c_int16

KIRO_TOOL_BDRATE = 9600

class KiroToolEmulator:
    def __init__(self, port='/dev/ttyS11', baudrate=KIRO_TOOL_BDRATE, timeout=1):
        self.sport = serial.Serial(port, baudrate, timeout=timeout, rtscts=True,dsrdtr=True)
        
    def recv_send(self):
        recv_str = self.sport.read_all()
        if len(recv_str)>0:
            recv_ints = str2ascii(recv_str)
            print(".".join(map(lambda x: "{:#04x}".format(x)[2:], recv_ints)))
            recv_ints[1] = (recv_ints[1] + 0xa0)%0x100
            self.sport.write(bytearray(recv_ints))

In [3]:
kte = KiroToolEmulator()

In [4]:
while True:
    kte.recv_send()
    time.sleep(0.5)

02.01.00.00.00.03.03
02.03.00.00.00.05.03
02.02.01.00.00.05.03
02.02.02.00.00.06.03
02.02.04.00.00.08.03
02.02.05.00.00.09.03
02.02.06.00.5a.64.03
02.02.06.00.a6.b0.03


KeyboardInterrupt: 